In [4]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [5]:
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [6]:
gpt4o_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'finish_reason': 'stop', 'logprobs': None}, id='run--c9a775c5-3477-4967-b81f-a9e81b74f9bf-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Load PDF
loader = PyPDFLoader("pdfs/bible.pdf")
docs = loader.load()

# Split text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

# Create vector store
vectorstore = Chroma.from_documents(split_docs, embedding=OpenAIEmbeddings(), persist_directory="./chroma_db")
vectorstore.persist()



/Users/claudio_hurtado/Code/clothing_agent/.venv/lib/python3.12/site-packages/numpy/linalg/_linalg.py:2791: RuntimeWarning: divide by zero encountered in dot
  sqnorm = x.dot(x)
/Users/claudio_hurtado/Code/clothing_agent/.venv/lib/python3.12/site-packages/numpy/linalg/_linalg.py:2791: RuntimeWarning: overflow encountered in dot
  sqnorm = x.dot(x)
/Users/claudio_hurtado/Code/clothing_agent/.venv/lib/python3.12/site-packages/numpy/linalg/_linalg.py:2791: RuntimeWarning: invalid value encountered in dot
  sqnorm = x.dot(x)
/var/folders/6m/qhplqbks3tn6vw3y5fxxxnr80000gn/T/ipykernel_7093/3305778719.py:16: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [7]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=gpt4o_chat,
    chain_type="stuff",  # or "map_reduce", "refine" depending on use case
    retriever=retriever,
    return_source_documents=True
)

In [12]:
query = "What is the most important teaching of this book?"
result = qa_chain({"query": query})

print("Answer:", result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(doc.metadata)

/Users/claudio_hurtado/Code/clothing_agent/.venv/lib/python3.12/site-packages/numpy/linalg/_linalg.py:2791: RuntimeWarning: divide by zero encountered in dot
  sqnorm = x.dot(x)
/Users/claudio_hurtado/Code/clothing_agent/.venv/lib/python3.12/site-packages/numpy/linalg/_linalg.py:2791: RuntimeWarning: overflow encountered in dot
  sqnorm = x.dot(x)
/Users/claudio_hurtado/Code/clothing_agent/.venv/lib/python3.12/site-packages/numpy/linalg/_linalg.py:2791: RuntimeWarning: invalid value encountered in dot
  sqnorm = x.dot(x)


Answer: The most important teaching highlighted in the provided context is to "Love the Lord your God with all your heart, with all your soul, with all your mind, and with all your strength" and to "Love your neighbor as yourself." These are described as the greatest and most important commands.

Sources:
{'trapped': '/False', 'page': 909, 'page_label': '900', 'source': 'pdfs/bible.pdf', 'total_pages': 1120, 'creationdate': '2017-10-26T20:59:36-05:00', 'moddate': '2017-11-07T16:41:59-06:00', 'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 2017 (Macintosh)'}
{'page': 1061, 'total_pages': 1120, 'producer': 'Adobe PDF Library 15.0', 'source': 'pdfs/bible.pdf', 'creationdate': '2017-10-26T20:59:36-05:00', 'trapped': '/False', 'creator': 'Adobe InDesign CC 2017 (Macintosh)', 'page_label': '1052', 'moddate': '2017-11-07T16:41:59-06:00'}
{'source': 'pdfs/bible.pdf', 'creationdate': '2017-10-26T20:59:36-05:00', 'trapped': '/False', 'creator': 'Adobe InDesign CC 2017 (Macint